In [69]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [70]:
import requests
import pandas as pd
import json


In [71]:
# Substitua 'SUA_CHAVE_DE_API' pela sua chave de API
API_KEY = 'RGAPI-f996c09d-3987-4e3b-aaed-0fa0469d0c3b'

# Região do servidor (por exemplo, 'br1' para o servidor brasileiro)
REGION = 'br1'

# Nome de invocador (seu nome de jogador)
#SUMMONER_NAME = 'teteco25'

# URL base da API da Riot Games
BASE_URL = f'https://{REGION}.api.riotgames.com/lol'


In [72]:
listDivision = ['DIAMOND','EMERALD','PLATINUM','GOLD','SILVER','BRONZE','IRON']
listTier = ['I','II','III','IV']
#RANKED_SOLO_5x5

In [73]:
def filter_data(data):
    filtered_data = []
    for item in data:
        localized_names = item["localizedNames"].get("pt_BR")
        if localized_names:
            item["localizedNames"] = localized_names
            filtered_data.append(item)
    return filtered_data


In [74]:
def obter_id_invocador(SUMMONER_NAME):
    summoner_url = f'https://{REGION}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{SUMMONER_NAME}'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(summoner_url, headers=headers)

    if response.status_code == 200:
        match_history_data = response.json()
        return match_history_data['puuid']  # Retorna o ID do invocador
    else:
        print(f'Erro ao obter ID do invocador. Código de status: {response.status_code}')
        return None


In [75]:
def getConfigChallenge():
    summoner_url = f'https://{REGION}.api.riotgames.com/lol/challenges/v1/challenges/config'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(summoner_url, headers=headers)

    if response.status_code == 200:
        filtered_data = filter_data(response.json())
        return filtered_data
    else:
        print(f'Erro ao obter ID do invocador. Código de status: {response.status_code}')
        return None

In [76]:
def getChallengeById(idPartida):
    match_history_url = f'https://{REGION}.api.riotgames.com/lol/challenges/v1/player-data/{idPartida}'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(match_history_url, headers=headers)

    if response.status_code == 200:
        player_challenges_data = response.json()
        return player_challenges_data
    else:
        print(f'Erro ao obter histórico de partidas. Código de stasstus: {response.status_code}')
        return None

In [77]:
def getListNamesByElo(tier,division):
    list_player = f'https://{REGION}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page=1'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(list_player, headers=headers)

    if response.status_code == 200:
        player_challenges_data = response.json()
        return player_challenges_data
    else:
        print(f'Erro ao obter histórico de partidas. Código de stasstus: {response.status_code}')
        return None

In [78]:
if __name__ == '__main__':
    # Obter o ID do invocador
    account_id = obter_id_invocador()

    if account_id:
        # Obter os dados dos desafios de configuração
        dfConfig = pd.DataFrame(getConfigChallenge())

        # Obter os dados do desafio do usuário
        dfUserChallenge = getChallengeById(account_id)
        
        df = pd.DataFrame(dfUserChallenge['challenges'])
        
        dfConfig['name'] = dfConfig['localizedNames'].apply(lambda x: x.get('name', ''))
        dfConfig['description'] = dfConfig['localizedNames'].apply(lambda x: x.get('description', ''))
        dfConfig['shortDescription'] = dfConfig['localizedNames'].apply(lambda x: x.get('shortDescription', ''))
        
        # Fundir os DataFrames df e dfConfig com base na correspondência entre df['challengeId'] e dfConfig['id']
        df_merged = pd.merge(df, dfConfig[['id', 'name','shortDescription']], left_on='challengeId', right_on='id', how='left')
        df_merged = df_merged.drop(columns=['id'])
        # Iterar sobre os desafios do usuário e imprimir o nome do desafio
        for index, row in df_merged.iterrows():
            print(f"ID: {row['challengeId']}")
            print(f"ID: {row['level']}")
            print(f"ID: {row['name']}")
            print("=" * 50)
    else:
        print("Nenhum histórico de partidas encontrado.")

TypeError: obter_id_invocador() missing 1 required positional argument: 'SUMMONER_NAME'

In [79]:
dfListNamesByElo = pd.DataFrame(getListNamesByElo(listTier[0],listDivision[0]))
dfResult = pd.DataFrame(columns=['summonerName', 'summonerID','tier','division'])

In [84]:
dfListNamesByElo_top5 = dfListNamesByElo.head(5)

for index, row in dfListNamesByElo_top5.iterrows():
    dfListNamesByElo_top5.loc[index, 'puuid'] = obter_id_invocador(row['summonerName'])


In [85]:
dfListNamesByElo_top5

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,puuid
0,bb766fcd-6fb3-4a83-bb4f-f23ef9219958,RANKED_SOLO_5x5,DIAMOND,I,HKLedzO7qssURZr1yFOdZRY_O7tQQSBYxxd3d2jsY7bWqdI,DaryL,64,22,15,False,False,False,True,abU3BrdDWI_A3w4RDv17yjgCfGgjZS63vyLz2dFlc6uutL...
1,45491ebe-679c-4608-9c71-e5a7901ac758,RANKED_SOLO_5x5,DIAMOND,I,tBU4nJduBvo-7JQ-O6NycykEx-j7PwCr6HqvvK3lb023YOM,neuralplasticity,75,22,20,False,False,False,False,AMwrD5yBi2STCZIdeTwJH3so5Av2dHcW2zHXzaQHjX9PGD...
2,2b8aa521-d780-4dad-989d-4c859524d5ef,RANKED_SOLO_5x5,DIAMOND,I,p_mp24OoqDibzV2hezuoTwizdGwdV0rM2xqI7z1i1D71Uh8,Justiça Marielle,7,64,70,False,False,False,True,30fKnRdigMOQV9cDG1cGLlZTv3p9NR8-zer_zwmh_k6g5z...
3,d3f6f565-34ff-40ee-978f-d8e2fad07624,RANKED_SOLO_5x5,DIAMOND,I,dgN65VyzxZwp5_GZ1pBvEtR-O6LEKOK72wQ5uEMoqP0jSEM,Patinhuu,0,43,33,False,False,False,False,bamiZG9BjMtQ8CQIa7l33kvjwvWvngzoHVy2VVjAMDUlqS...
4,63e910fe-c49e-400e-bbe9-f797bf9dc735,RANKED_SOLO_5x5,DIAMOND,I,QUpaShwawcc6DqxoBKoeFLhS8_wN7FXRft05oFRsuqozdLM,Fallen of Albaz,21,25,13,False,False,False,False,_x2axTUDQuMjMM6AqnROh3TTjWCWTqtT67sKYEyLCnW2hF...


In [88]:
df_concat = pd.DataFrame()
for index, row in dfListNamesByElo_top5.iterrows():
    # Obtendo o nome do invocador da linha atual
    puuid = row['puuid']
    tier = row['tier']
    division = row['rank']
    summonerName = row['summonerName']
    dfUserChallenge = getChallengeById(puuid)
    
    df = pd.DataFrame(dfUserChallenge['challenges'])

    dfConfig = pd.DataFrame(getConfigChallenge())
    dfConfig['name'] = dfConfig['localizedNames'].apply(lambda x: x.get('name', ''))
    dfConfig['description'] = dfConfig['localizedNames'].apply(lambda x: x.get('description', ''))
    dfConfig['shortDescription'] = dfConfig['localizedNames'].apply(lambda x: x.get('shortDescription', ''))
    
    # Fundir os DataFrames df e dfConfig com base na correspondência entre df['challengeId'] e dfConfig['id']
    df_merged = pd.merge(df, dfConfig[['id', 'name','shortDescription']], left_on='challengeId', right_on='id', how='left')
    df_merged = df_merged.drop(columns=['id'])
    

    df_new = df_merged.set_index('name')['level'].to_frame().T
    df2 = df_new.append({'summonerName': summonerName, 'puuid': puuid, 'tier': tier, 'division': division}, ignore_index=True)
    df_concat = pd.concat([df2, df_concat])

AttributeError: 'DataFrame' object has no attribute 'append'

In [98]:

# DataFrame maior (exemplo)
df_maior = pd.DataFrame({
    'grupo': ['A', 'A', 'B', 'B'],
    'categoria': ['x', 'y', 'x', 'y'],
    'valor': [1, 2, 3, 4]
})

# DataFrame com as linhas que você deseja transformar em colunas
df_pivot = pd.DataFrame({
    'grupo': ['A', 'B', 'A', 'B'],
    'linha': ['linha1', 'linha2', 'linha3', 'linha4'],
    'valor': [10, 20, 30, 40]
})
# Pivotar o DataFrame df_pivot
df_pivot_pivotado = df_pivot.pivot(index='grupo', columns='linha', values='valor')

# Converter cada linha do DataFrame pivotado em um dicionário
dict_pivotado = df_pivot_pivotado.to_dict(orient='records')[0]

# Criar um novo DataFrame com uma única linha
df_pivot_pivotado_transposto = pd.DataFrame(dict_pivotado, index=[0])



In [99]:
df_pivot_pivotado_transposto

,linha1,linha2,linha3,linha4
0,10.0,NaN,30.0,NaN


In [100]:
df_merged

,challengeId,percentile,level,value,achievedTime,position,playersInLevel,name,shortDescription
0,0,0.067,PLATINUM,12220.0,1.693509e+12,NaN,NaN,CRISTAL,Regras especiais para o Cristal
1,1,0.065,PLATINUM,1260.0,1.705092e+12,NaN,NaN,IMAGINAÇÃO,Divisão IMAGINAÇÃO
2,2,0.092,PLATINUM,3080.0,1.673913e+12,NaN,NaN,ESPECIALIZAÇÃO,Divisão ESPECIALIZAÇÃO
3,3,0.097,PLATINUM,2650.0,1.663112e+12,NaN,NaN,VIVÊNCIA,Divisão VIVÊNCIA
4,4,0.064,PLATINUM,3240.0,1.693509e+12,NaN,NaN,TRABALHO EM EQUIPE,Divisão TRABALHO EM EQUIPE
...,...,...,...,...,...,...,...,...,...
303,402403,0.090,PLATINUM,743.0,1.694934e+12,NaN,NaN,Adeus à Escuridão,Posicione Sentinelas de Controle
304,402400,0.077,DIAMOND,520.0,1.695232e+12,NaN,NaN,Engenhosidade,Ganhe pontos nos desafios da categoria Engenho...
305,402401,0.089,DIAMOND,1821.0,1.695945e+12,NaN,NaN,Escuridão Absoluta,Elimine sentinelas
306,402408,0.069,DIAMOND,289850.0,1.694205e+12,NaN,NaN,Todo Mundo Paga!,Ganhe recompensas em ouro


In [ ]:
df_concat

In [ ]:
dfUserChallenge

In [101]:
# Pivotar o DataFrame
df_pivoted = df_merged.pivot(index='level', columns='name', values='value').reset_index()

# Redefinir o índice para uma única linha
df_pivoted = df_pivoted.iloc[[0]]
df_pivoted

name,level,''Ultimate'' Já Diz Tudo,A morte foge de mim: 2023,AMA Invejável,Abuso de Habilidade,Ace,Ace Impecável,Acerto Certeiro,Acima de todos: 2023,Acumulando Acúmulos,...,Vigia de Sentinelas,Virtuoso,Visionário,Visual Vintage,Vitória Declarada,Vitória Perfeita,Vou me esbaldar: 2023,Vá Aonde Quiser,\nFlame Horizon,Ícone do Rift
0,BRONZE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
